In [7]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import smtplib #send email
import pandas as pd
import json

In [101]:
url = 'https://eu-site-api.byd.com/byd-api/eu/dealer/getFindDealer?country=DE&type=sales'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
page = requests.get(url, headers = headers)

soup1 = BeautifulSoup(page.text, 'html.parser')
soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

In [103]:
json_text = soup2.get_text()

data = json.loads(json_text)
df = pd.json_normalize(data)

In [37]:
pd.set_option('display.max_columns', None)

In [38]:
df = df[['country','organizationName','lng', 'lat','zipCode','phoneNumber','feature', 'address',]]

In [46]:
df = df.copy()
df['timestamp'] = pd.to_datetime('now')

In [9]:
import os
from time import time
from time import sleep
from datetime import date

In [15]:
def dealer_point_runner(country_list):
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    for country in country_list:
        url = f'https://eu-site-api.byd.com/byd-api/eu/dealer/getFindDealer?country={country}&type=sales'
        try:
            page = requests.get(url, headers = headers)
            soup1 = BeautifulSoup(page.text, 'html.parser')
            soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

            json_text = soup2.get_text()
            data = json.loads(json_text)
            df = pd.json_normalize(data)

            df = df[['country','organizationName','lng', 'lat','zipCode','phoneNumber','feature', 'address',]]
            df = df.copy()
            df['timestamp'] = date.today()

            # correct the misordered input location data
            condition = (df['lng'] > 40) & (df['lat'] < 30)
            df = df.copy()
            df.loc[condition, ['lng', 'lat']] = df.loc[condition, ['lat', 'lng']].values

            if not os.path.isfile(r'D:\DS_road\data\web_scraping\BYD_eunew.csv'):
                df.to_csv(r'D:\DS_road\data\web_scraping\BYD_eunew.csv',header = 'column_names')
            else:
                df.to_csv(r'D:\DS_road\data\web_scraping\BYD_eunew.csv',mode = 'a',header = False)
            print(f"Successfully scraped {country}")

        except Exception as e:
            print(f"Failed for {country}: {e}")

In [17]:
country_codes = ['BE','BG','CZ','DK','DE','EE','IE','EL','ES','FR','HR','IT','CY','LV','LT','LU','HU','MT','NL','AT','PL','PT','RO','SI','SK','FI','SE','NO','CH']

for i in range(333):
    
    dealer_point_runner(country_codes)
    print("one rund completed")
    sleep(60)  #432000 = 5days
exit()

Successfully scraped BE
Failed for BG: "None of [Index(['country', 'organizationName', 'lng', 'lat', 'zipCode', 'phoneNumber',\n       'feature', 'address'],\n      dtype='object')] are in the [columns]"
Successfully scraped CZ
Successfully scraped DK
Successfully scraped DE
Successfully scraped EE
Successfully scraped IE
Failed for EL: "None of [Index(['country', 'organizationName', 'lng', 'lat', 'zipCode', 'phoneNumber',\n       'feature', 'address'],\n      dtype='object')] are in the [columns]"
Successfully scraped ES
Successfully scraped FR
Successfully scraped HR
Successfully scraped IT
Failed for CY: "None of [Index(['country', 'organizationName', 'lng', 'lat', 'zipCode', 'phoneNumber',\n       'feature', 'address'],\n      dtype='object')] are in the [columns]"
Successfully scraped LV
Successfully scraped LT
Successfully scraped LU
Successfully scraped HU
Failed for MT: "None of [Index(['country', 'organizationName', 'lng', 'lat', 'zipCode', 'phoneNumber',\n       'feature', 'a

KeyboardInterrupt: 